In [1]:
from agent_nodes import g2i

import rospy
from context_based_gesture_operation.srv import G2I, G2IResponse
from context_based_gesture_operation.msg import Intent
from context_based_gesture_operation.msg import Scene as SceneRos
from context_based_gesture_operation.msg import Gestures as GesturesRos
import numpy as np
from srcmodules.Scenes import Scene
from srcmodules.Gestures import Gestures
from srcmodules.Actions import Actions
from srcmodules.SceneFieldFeatures import SceneFieldFeatures

#  Behaviour tree example

- `roscore` need to be running in the backend
- `rosrun rqt_py_trees rqt_py_trees` to visualize the tree

### Init Gestures to Intent Service

In [2]:
A = ['move_up', 'move_left', 'move_down', 'move_right',
         'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']
G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right',
          'five', 'grab', 'thumbsup', 'rotate', 'point', 'two', 'three']

In [3]:
g2i.G2IService(G, A)

### Init scene & gesture vector publishers

In [4]:
pub_scene = rospy.Publisher("/tree/scene_in", SceneRos, queue_size=5)
pub_gestures = rospy.Publisher("/tree/gestures_in", GesturesRos, queue_size=5)

### Save the robotic actions

In [5]:
robotic_sequence = []
def save_the_robotic_sequence(msg):
    global robotic_sequence
    robotic_sequence.append(msg)
rospy.Subscriber('/execute_intent', Intent, save_the_robotic_sequence)

### Create B Tree

In [6]:
from srcmodules import BTreeLib
import py_trees_ros, functools

In [7]:
rospy.init_node("tree")

root = BTreeLib.create_tree()
behaviour_tree = py_trees_ros.trees.BehaviourTree(root)
rospy.on_shutdown(functools.partial(BTreeLib.shutdown, behaviour_tree))

if not behaviour_tree.setup(timeout=15):
    console.logerror("failed to setup the tree, aborting.")
    sys.exit(1)

In [8]:
behaviour_tree.tick()

update scene: Status.RUNNING


# Ex. 1 
- put to drawer, when drawer is closed

1. Generate Scene and Gesture vector
2. Publish Scene and gesture vector info and generate the robotic action sequence 

In [9]:
g = GesturesRos()
g.probabilities.data = np.zeros(11)
g.probabilities.data[5] = 1.0

s = Scene(init='object,drawer,cup', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = s.drawer.position

In [10]:
pub_scene.publish(sr)
pub_gestures.publish(g)

In [11]:
behaviour_tree.tick()

update scene: Status.SUCCESS
update gestures: Status.SUCCESS
intent put_on, drawer
generate intent: Status.SUCCESS
Execute TA Status.SUCCESS! put_on,drawer


[WARN] [1665055571.135344]: Scene2BB: gripper attached!


In [12]:
robotic_sequence

[target_action: "place"
 target_object: ''
 auxiliary_parameters: '',
 target_action: "open"
 target_object: "drawer"
 auxiliary_parameters: '',
 target_action: "pick_up"
 target_object: "cup"
 auxiliary_parameters: '',
 target_action: "put_on"
 target_object: "drawer"
 auxiliary_parameters: '']

# Ex. 2
- move left

In [14]:
robotic_sequence = []

g = GesturesRos()
g.probabilities.data = np.zeros(11)
g.probabilities.data[1] = 1.0

s = Scene(init='object,drawer,cup', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = s.object.position

pub_scene.publish(sr)
pub_gestures.publish(g)

behaviour_tree.tick()
robotic_sequence

update scene: Status.SUCCESS
update gestures: Status.SUCCESS
intent move_left, object
generate intent: Status.SUCCESS
Execute TA Status.SUCCESS! move_left,object


[target_action: "move_left"
 target_object: "object"
 auxiliary_parameters: '']

# Ex. 3
- put to drawer, drawer is opened

In [16]:
robotic_sequence = []

g = GesturesRos()
g.probabilities.data = np.zeros(11)
g.probabilities.data[5] = 1.0

s = Scene(init='object,drawer,cup', random=False)
s.drawer.open()
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = s.drawer.position

pub_scene.publish(sr)
pub_gestures.publish(g)

behaviour_tree.tick()
robotic_sequence

update scene: Status.SUCCESS
update gestures: Status.SUCCESS
intent put_on, drawer
generate intent: Status.SUCCESS
Execute TA Status.SUCCESS! put_on,drawer


[target_action: "put_on"
 target_object: "drawer"
 auxiliary_parameters: '']

# Ex. 4
- put to drawer, not holding anything, target drawer opened
    
- Unfeasible, no object held in gripper -> action discarded before when checking action validity 